In [1]:
from pathlib import Path
import subprocess
import pandas as pd
import json, pickle,sys,glob
import simplekml, xmltodict
from shapely.geometry import Polygon, Point
import yaml
from mymodules.utils import *
from mymodules.myscripts import kml_mapper
from mymodules.aisight.naqsha import *

In [2]:
# base directory
BASE_DIR = Path().parent.resolve()

with open("config.yaml", "r") as yf:
    config = yaml.safe_load(yf)

# just to add collect script available on path
sys.path.append(Path(config["paths"]["mymodules"]).resolve())

In [3]:
# reading config from yaml
company = config["analysis"]["company"]
country = config["analysis"]["country"]
area_name = config["analysis"]["area"]

# output folder
# output_folder = Path(config["paths"]["data"]["output"]).resolve()

In [5]:
# update paths section
config['paths'] = replace_placeholders(config['paths'], company, country)

print(yaml.dump(config, default_flow_style=False))

analysis:
  area: 6731dc27c09ec86f8546f091
  company: bat
  country: czech
  exclude: []
geo:
  tif:
    scale: 1.0
paths:
  data: projects/bat/czech/data
  date:
    current_month: 2024-09-30
  logs: projects/bat/czech/logs
  mymodules: mymodules
  output: projects/bat/czech/output
  raw: projects/bat/czech/raw
process:
  debug: 0
  multiprocessing: 1



In [8]:
geo_dir = Path(config["paths"]['data']).resolve() / area_name
geo_dir

WindowsPath('D:/Office/rtm/projects/bat/czech/data/6731dc27c09ec86f8546f091')

In [9]:
boundary, ptif, sec, whitespace = kml_mapper.setup(geo_dir)

Tif Dimensions:  Width: 251  ||  Height: 120
Caching & Converting Tif Pixels . . . . .


100%|██████████| 120/120 [00:00<00:00, 692.52it/s]


Propagating SEC . . .
Propagating SEC . . .


In [ ]:
'''
get intersected box of tif with boundry
'''

In [24]:
boundary._polygons[0]
TIF_data = ptif.to_dict(boundary._polygons[0])
TIF_data[0]

{'position': (119, 165),
 'population': 7.0,
 'coordinates': [Coordinates(longitude=16.924999999999216, latitude=48.633333333332786),
  Coordinates(longitude=16.924999999999216, latitude=48.62499999999945),
  Coordinates(longitude=16.93333333333255, latitude=48.62499999999945),
  Coordinates(longitude=16.93333333333255, latitude=48.633333333332786),
  Coordinates(longitude=16.924999999999216, latitude=48.633333333332786)]}

In [ ]:
boundary_polygon = boundary._polygons[0]
TIF_data = ptif.to_dict()
output_dict = []
for i in TIF_data:
    coordinates = TIF_data[i]['coordinates']
    population = TIF_data[i]['population']
    pixel_poly = Polygon(coordinates)
    area_percent = (
        pixel_poly.intersection(boundary_polygon).area / pixel_poly.area
    )

    name=round(population*area_percent)
    name_initial = int(name)
    name = (name_initial / 100) * 255
    name = hex(int(name)).split('x')[-1].zfill(2)
    
    if name_initial > 100: color = f'ff0000ff'
    else: color = f'ff0000{name}'
    output_dict.append({
        "coordinates": [
            {"lat": i[1], "lng": i[0]} for i in coordinates
        ],
        "population" : population,
        "color" : color
    })

{0: {'position': (0, 120),
  'population': 3.0,
  'coordinates': [Coordinates(longitude=16.549999999999216, latitude=49.62499999999945),
   Coordinates(longitude=16.549999999999216, latitude=49.61666666666612),
   Coordinates(longitude=16.55833333333255, latitude=49.61666666666612),
   Coordinates(longitude=16.55833333333255, latitude=49.62499999999945),
   Coordinates(longitude=16.549999999999216, latitude=49.62499999999945)]},
 1: {'position': (0, 121),
  'population': 33.0,
  'coordinates': [Coordinates(longitude=16.55833333333255, latitude=49.62499999999945),
   Coordinates(longitude=16.55833333333255, latitude=49.61666666666612),
   Coordinates(longitude=16.56666666666588, latitude=49.61666666666612),
   Coordinates(longitude=16.56666666666588, latitude=49.62499999999945),
   Coordinates(longitude=16.55833333333255, latitude=49.62499999999945)]},
 2: {'position': (0, 122),
  'population': 4.0,
  'coordinates': [Coordinates(longitude=16.56666666666588, latitude=49.62499999999945),
